In [ ]:
"""Compute chrY z-scores with regards to different distributions, and plot the results."""

# pylint: disable=line-too-long, redefined-outer-name, import-error, pointless-statement, use-dict-literal, expression-not-assigned, unused-import, too-many-lines, unreachable, duplicate-code
from __future__ import annotations

from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots
from scipy.signal import find_peaks
from scipy.stats import gaussian_kde

from epiclass.core.metadata import Metadata
from epiclass.utils.general_utility import get_valid_filename

## Setup

In [ ]:
ASSAY = "assay_epiclass"
SEX = "harmonized_donor_sex"
TRACK = "track_type"

In [ ]:
epilap_project = Path.home() / "Projects/epilap"

metadata = (
    epilap_project
    / "input"
    / "metadata"
    / "dfreeze-v2"
    / "hg38_2023-epiatlas-dfreeze_v2.1_w_encode_noncore_2.json"
)

logs = epilap_project / "output/logs/epiatlas-dfreeze-v2.1"
chrY_coverage_file = logs / "chrY_coverage_results" / "chrXY_coverage_all.csv"

In [ ]:
meta = Metadata(metadata)

In [ ]:
df_chrY = pd.read_csv(chrY_coverage_file, index_col=0, header=0)

In [ ]:
print(df_chrY.columns)

## Merge metadata and chrY coverage info

In [ ]:
meta_df = pd.DataFrame.from_records(list(meta.datasets)).set_index("md5sum")

In [ ]:
meta_df[ASSAY].value_counts()

In [ ]:
# remove non-core tracks
meta_df = meta_df[~meta_df[ASSAY].str.contains("non-core|CTCF")]

In [ ]:
merged_df = df_chrY.merge(meta_df, left_index=True, right_index=True)

In [ ]:
print(meta_df.shape, df_chrY.shape, merged_df.shape)

In [ ]:
# Check the merge
assert (merged_df.shape[0] == meta_df.shape[0]) and (
    df_chrY.shape[1] + meta_df.shape[1] == merged_df.shape[1]
), "not right shape"

In [ ]:
# for item in set(meta_df.index.values) - set(df_chrY.index.values):
#     print(item)

In [ ]:
# merged_df

## Compute chrY zscore coverage by assay and sex

In [ ]:
# groupby_columns = [ASSAY, SEX, TRACK]
# groupby_columns = [ASSAY, SEX]
groupby_columns = [ASSAY, TRACK]

chrY_dists = merged_df.groupby(groupby_columns).agg({"chrY": ["mean", "std", "count"]})
display(chrY_dists)
# display(chrY_dists.index)

In [ ]:
partial_name = "assay_track"

In [ ]:
chrY_dists.to_csv(
    logs / "chrY_coverage_results" / f"chrY_coverage_distributions_{partial_name}.csv"
)

In [ ]:
score_name = f"expected_{partial_name}_chrY_z-score"

In [ ]:
new_data = []
for row in chrY_dists.iterrows():
    index, vals = row
    # assay, sex, track = index # type: ignore
    # assay, sex = index # type: ignore
    # assay = index  # type: ignore
    assay, track = index  # type: ignore

    mean = vals["chrY"]["mean"]
    std = vals["chrY"]["std"]
    count = vals["chrY"]["count"]

    # partial_df = merged_df[(merged_df[ASSAY] == assay) & (merged_df[SEX] == sex) & (merged_df[TRACK] == track)]
    # partial_df = merged_df[(merged_df[ASSAY] == assay) & (merged_df[SEX] == sex)]
    # partial_df = merged_df[(merged_df[ASSAY] == assay)]
    partial_df = merged_df[(merged_df[ASSAY] == assay) & (merged_df[TRACK] == track)]
    partial_df = partial_df[["chrY"]].copy()

    partial_df.loc[:, score_name] = (partial_df["chrY"] - mean) / std
    partial_df.loc[:, f"count_{score_name}"] = count

    new_data.append(partial_df)

In [ ]:
# print(new_data)

In [ ]:
all_zscores_df = pd.concat(new_data)

In [ ]:
final_df = df_chrY.merge(
    all_zscores_df, how="left", left_index=True, right_index=True, suffixes=("", "_DROP")
).sort_index()
final_df = final_df.drop(columns=["chrY_DROP"])

In [ ]:
final_df_path = (
    logs / "chrY_coverage_results" / f"chrY_coverage_zscores_{partial_name}.csv"
)
# final_df.to_csv(final_df_path)

## Map predictions to chrY z-scores

In [ ]:
sex_pred_dir = Path.home() / "downloads" / "temp"
sex_pred_file = (
    sex_pred_dir / "sex3_oversample_full-10fold-validation_prediction_augmented-all.csv"
)

full_chrY_df = pd.read_csv(final_df_path, index_col=0, header=0)

In [ ]:
pred_df = pd.read_csv(sex_pred_file, index_col=0, header=0)
print(pred_df.shape)

In [ ]:
merged_pred_df = pred_df.merge(
    full_chrY_df, how="inner", left_index=True, right_index=True, suffixes=("", "_DROP")
).sort_index()
merged_pred_df = merged_pred_df.drop(
    columns=[column for column in pred_df.columns if column.endswith("_DROP")]
)
print(merged_pred_df.shape)

In [ ]:
raise UserWarning("stop here")

## chrY z-scores confusion matrix for sex3 preds

In [ ]:
COLORS_DICT = {"female": "red", "male": "blue", "mixed": "purple"}

In [ ]:
# merged_pred_df.to_csv(sex_pred_dir / "sex3_oversample_full-10fold-validation_prediction_augmented-all_chrY_zscores.csv")

In [ ]:
# print([col for col in merged_pred_df.columns if "z-score" in col])

In [ ]:
# merged_pred_df = merged_pred_df[
#     ~merged_pred_df[ASSAY].str.contains(pat="wgb", case=False)
# ]

### graphs per assay

In [ ]:
coverage_label = "expected_assay_chrY_z-score"
count_label = f"count_{coverage_label}"

classes = merged_pred_df["True class"].unique()
assays = merged_pred_df[ASSAY].unique()

matrix_logdir = (
    logs
    / "chrY_coverage_results"
    / "10fold_valid"
    / "z-score"
    / "per_assay"
    / "w-unknown"
    / "per_assay_graph"
)

for assay_label in assays:
    assay_df = merged_pred_df[merged_pred_df[ASSAY] == assay_label]

    # confusion matrix for chrY z-score
    for threshold in [0, 0.7, 0.9]:
        row = 1
        col = 1
        fig = make_subplots(
            rows=3,
            cols=3,
            shared_yaxes=True,
            x_title="Predicted class (nb of predictions)",
            y_title="z-score vs expected assay",
            row_titles=list(classes),
            column_titles=list(classes),
            vertical_spacing=0.08,
            horizontal_spacing=0.01,
        )
        threshold_df = assay_df[assay_df["Max pred"] >= threshold]

        for label in classes:
            df_label = threshold_df[threshold_df["True class"] == label]

            # Iterate over each target and add a violin plot for it
            for target in classes:
                sub_df = df_label[df_label["Predicted class"] == target]

                if sub_df.shape[0] == 0:
                    y_values = [
                        threshold_df[coverage_label].mean()
                    ]  # Minimal synthetic data
                    sample_count = 0
                    hovertext = ["PLACEHOLDER - NO DATA"]
                else:
                    y_values = sub_df[coverage_label]
                    hovertext = [
                        f"{md5sum, assay}:(z-score={z_score:.3f} (n={int(count)}), pred={pred:.3f})"
                        for md5sum, pred, z_score, count, assay in zip(
                            sub_df.index,
                            sub_df["Max pred"],
                            sub_df[coverage_label],
                            sub_df[count_label],
                            sub_df[ASSAY],
                        )
                    ]

                fig.add_trace(
                    go.Violin(
                        y=y_values,
                        name=f"{target} ({sub_df.shape[0]})",
                        box_visible=True,
                        meanline_visible=True,
                        points="all",
                        line_color=COLORS_DICT[target],
                        hovertemplate="%{text}",
                        text=hovertext,
                    ),
                    row=row,
                    col=col,
                )

                # Move to the next subplot position
                col += 1
                if col > 3:
                    col = 1
                    row += 1

        # Update global layout and traces
        fig.update_traces(marker=dict(size=1))
        fig.update_yaxes(
            range=[
                min(assay_df[coverage_label]) - 0.01,
                max(assay_df[coverage_label]) + 0.01,
            ]
        )

        # Directly using annotations param does not work with make_subplots
        existing_annotations = fig.layout.annotations
        new_annotation = dict(
            x=1.01,  # Position on the x-axis
            y=0.5,  # Position on the y-axis
            showarrow=False,  # Do not show arrow
            text="Reference class",  # The text you want to display
            xref="paper",  # 'x' coordinate is set in relative coordinates
            yref="paper",  # 'y' coordinate is set in relative coordinates
            xanchor="left",  # Text starts from the left of the x-coordinate
            yanchor="middle",  # Middle aligned vertically
            font=dict(size=16),
            textangle=90,
        )
        updated_annotations = list(existing_annotations) + [new_annotation]

        title = f"z-score(mean chrY coverage per file):{assay_label} (pred>{threshold})"

        fig.update_layout(
            title_text=f"{title} (n={threshold_df.shape[0]})",
            showlegend=False,
            annotations=updated_annotations,
        )

        # fig.show()

        # title = get_valid_filename(title).replace("_br_", "_")
        # html_file = matrix_logdir / f"{title}.html"
        # png_file = matrix_logdir / f"{title}.png"
        # if not png_file.exists():
        #     fig.write_image(png_file, scale=2)
        # if not html_file.exists():
        #     fig.write_html(html_file)

### Graphs per assay and track

In [ ]:
sex_df = merged_pred_df[merged_pred_df[SEX].notnull()]

In [ ]:
partial_name = "assay_track"
coverage_label = f"expected_{partial_name}_chrY_z-score"
count_label = f"count_{coverage_label}"

classes = sex_df[SEX].unique()
assays = sex_df[ASSAY].unique()

In [ ]:
print(classes, assays)

In [ ]:
matrix_logdir = (
    logs
    / "chrY_coverage_results"
    / "10fold_valid"
    / "z-score"
    / "per_assay_track"
    / "per_assay_track_confusion_graphs"
)
matrix_logdir.mkdir(parents=False, exist_ok=True)

#### Confusion matrices style

In [ ]:
for assay_label in assays:
    assay_df = merged_pred_df[merged_pred_df[ASSAY] == assay_label]

    for track_type in assay_df[TRACK].unique():
        assay_track_df = assay_df[assay_df[TRACK] == track_type]

        # confusion matrix for chrY z-score
        for threshold in [0, 0.7, 0.9]:
            row = 1
            col = 1
            fig = make_subplots(
                rows=3,
                cols=3,
                shared_yaxes=True,
                x_title="Predicted class (nb of predictions)",
                y_title="z-score vs expected assay",
                row_titles=list(classes),
                column_titles=list(classes),
                vertical_spacing=0.08,
                horizontal_spacing=0.01,
            )
            threshold_df = assay_track_df[assay_track_df["Max pred"] >= threshold]

            title = f"z-score(mean chrY coverage per file):{assay_label},{track_type} (pred>{threshold})"
            filename = get_valid_filename(title).replace("_br_", "_")

            html_file = matrix_logdir / f"{filename}.html"
            png_file = matrix_logdir / f"{filename}.png"
            if png_file.exists() or html_file.exists():
                continue

            for label in classes:
                df_label = threshold_df[threshold_df["True class"] == label]

                # Iterate over each target and add a violin plot for it
                for target in classes:
                    sub_df = df_label[df_label["Predicted class"] == target]

                    if sub_df.shape[0] == 0:
                        y_values = [
                            threshold_df[coverage_label].mean()
                        ]  # Minimal synthetic data
                        sample_count = 0
                        hovertext = ["PLACEHOLDER - NO DATA"]
                    else:
                        y_values = sub_df[coverage_label]
                        hovertext = [
                            f"{md5sum}:(z-score={z_score:.3f} (n={int(count)}), pred={pred:.3f})"
                            for md5sum, pred, z_score, count, assay in zip(
                                sub_df.index,
                                sub_df["Max pred"],
                                sub_df[coverage_label],
                                sub_df[count_label],
                                sub_df[ASSAY],
                            )
                        ]

                    fig.add_trace(
                        go.Violin(
                            y=y_values,
                            name=f"{target} ({sub_df.shape[0]})",
                            box_visible=True,
                            meanline_visible=True,
                            points="all",
                            line_color=COLORS_DICT[target],
                            hovertemplate="%{text}",
                            text=hovertext,
                        ),
                        row=row,
                        col=col,
                    )

                    # Move to the next subplot position
                    col += 1
                    if col > 3:
                        col = 1
                        row += 1

            # Update global layout and traces
            fig.update_traces(marker=dict(size=1))
            fig.update_yaxes(
                range=[
                    min(assay_track_df[coverage_label]) - 0.01,
                    max(assay_track_df[coverage_label]) + 0.01,
                ]
            )

            # Directly using annotations param does not work with make_subplots
            existing_annotations = fig.layout.annotations
            new_annotation = dict(
                x=1.01,  # Position on the x-axis
                y=0.5,  # Position on the y-axis
                showarrow=False,  # Do not show arrow
                text="Reference class",  # The text you want to display
                xref="paper",  # 'x' coordinate is set in relative coordinates
                yref="paper",  # 'y' coordinate is set in relative coordinates
                xanchor="left",  # Text starts from the left of the x-coordinate
                yanchor="middle",  # Middle aligned vertically
                font=dict(size=16),
                textangle=90,
            )
            updated_annotations = list(existing_annotations) + [new_annotation]

            fig.update_layout(
                title_text=f"{title} (n={threshold_df.shape[0]})",
                showlegend=False,
                annotations=updated_annotations,
            )

            # fig.show()
            # break
            # fig.write_image(png_file, scale=2)
            # fig.write_html(html_file)

#### Global assay/track distributions

In [ ]:
sex_df[ASSAY].value_counts()

In [ ]:
matrix_logdir = (
    logs / "chrY_coverage_results" / "10fold_valid" / "z-score" / "per_assay_track"
)

if not matrix_logdir.exists():
    matrix_logdir.mkdir(parents=False, exist_ok=True)

# Prepare a subplot grid; one for each assay + track type combination
num_assay_track_combinations = sum(
    len(sex_df[sex_df[ASSAY] == assay_label][TRACK].unique()) for assay_label in assays
)
total_cols = num_assay_track_combinations + len(sex_df[TRACK].unique())

In [ ]:
assay_colors_dict = dict(zip(assays, px.colors.qualitative.Dark24))
track_colors_dict = dict(zip(sex_df[TRACK].unique(), px.colors.qualitative.Dark24_r))

#### z-score violins - any sex

In [ ]:
col = 1
row = 1

x_title = "Assay+Track distribution"
fig = make_subplots(
    rows=row,
    cols=total_cols,
    shared_yaxes=True,
    x_title=x_title,
    y_title="z-score",
    horizontal_spacing=0.001,
)

for assay_label in sorted(assays):
    assay_df = merged_pred_df[merged_pred_df[ASSAY] == assay_label]

    for track_type in sorted(assay_df[TRACK].unique()):
        sub_df = assay_df[assay_df[TRACK] == track_type]

        y_values = sub_df[coverage_label]
        hovertext = [
            f"{md5sum}:z-score={z_score:.3f} (n={int(count)}), pred={pred:.3f}"
            for md5sum, pred, z_score, count in zip(
                sub_df.index,
                sub_df["Max pred"],
                sub_df[coverage_label],
                sub_df[count_label],
            )
        ]

        fig.add_trace(
            go.Violin(
                y=y_values,
                name=f"{assay_label},{track_type} ({sub_df.shape[0]})",
                box_visible=True,
                meanline_visible=True,
                points="all",
                line_color=assay_colors_dict[assay_label],
                hovertemplate="%{text}",
                text=hovertext,
            ),
            row=row,
            col=col,
        )

        col += 1


# global track type distribution (all assays)
for track_type in sorted(merged_pred_df[TRACK].unique()):
    sub_df = merged_pred_df[merged_pred_df[TRACK] == track_type]

    y_values = sub_df[coverage_label]
    hovertext = [
        f"{md5sum,assay}:z-score={z_score:.3f} (n={int(count)}), pred={pred:.3f}"
        for md5sum, pred, z_score, count, assay in zip(
            sub_df.index,
            sub_df["Max pred"],
            sub_df[coverage_label],
            sub_df[count_label],
            sub_df[ASSAY],
        )
    ]

    fig.add_trace(
        go.Violin(
            y=y_values,
            name=f"{track_type} ({sub_df.shape[0]})",
            box_visible=True,
            meanline_visible=True,
            points="all",
            line_color=track_colors_dict[track_type],
            hovertemplate="%{text}",
            text=hovertext,
        ),
        row=row,
        col=col,
    )

    col += 1

# Final graphing

# Update global layout and traces
fig.update_traces(marker=dict(size=1))
fig.update_yaxes(
    range=[
        min(merged_pred_df[coverage_label]) - 0.01,
        max(merged_pred_df[coverage_label]) + 0.01,
    ]
)

fig.update_xaxes(tickangle=50)

fig.update_annotations(y=1.5, selector={"text": x_title})

title = "z-score(mean chrY coverage per file) distribution per assay+track type"
fig.update_layout(
    title_text=f"{title}",
    showlegend=False,
    autosize=True,
    width=3000,
)

# fig.show()

# for violin in fig.data:
#     print(dir(violin))
#     print(violin)
#     break

# filename = get_valid_filename(title).replace("_br_", "_")
# html_file = matrix_logdir / f"{filename}.html"
# png_file = matrix_logdir / f"{filename}.png"

# fig.write_image(png_file, scale=3)
# fig.write_html(html_file)

#### z-score violins - split sex

In [ ]:
partial_name = "assay_track"
coverage_label = f"expected_{partial_name}_chrY_z-score"
count_label = f"count_{coverage_label}"

In [ ]:
all_results_file = (
    logs
    / "merged_results"
    / "epiatlas"
    / "with_split_nb"
    / "merged_pred_results_all_2.1_chrY_zscores.csv"
)
merged_pred_df = pd.read_csv(all_results_file, index_col=0, header=0, low_memory=False)

In [ ]:
total_cols = num_assay_track_combinations * 2 + len(merged_pred_df[TRACK].unique())

to_verify_epirr = [
    "IHECRE00004623",
    "IHECRE00000171",
    "IHECRE00001957",
    "IHECRE00000152",
    "IHECRE00001531",
    "IHECRE00000951",
    "IHECRE00001965",
    "IHECRE00000099",
    "IHECRE00000316",
    "IHECRE00004877",
    "IHECRE00003706",
    "IHECRE00001370",
    "IHECRE00001001",
    "IHECRE00000954",
    "IHECRE00004890",
]

In [ ]:
[col for col in merged_pred_df.columns if "epirr" in str(col).lower()]

In [ ]:
merged_pred_df["epirr_no_version"] = merged_pred_df["EpiRR"].str.split(".").str[0]

In [ ]:
merged_pred_df[merged_pred_df["epirr_no_version"].isin(to_verify_epirr)].shape

In [ ]:
epirr_cat = "epirr_no_version"
black = "#000000"
white = "#FFFFFF"

In [ ]:
epirr_colors = dict(zip(to_verify_epirr, px.colors.qualitative.Dark24))

In [ ]:
sex_df.shape

In [ ]:
sex_df.harmonized_donor_sex.value_counts()

In [ ]:
col = 1
row = 1
x_title = "Assay+Track distribution"
fig = make_subplots(
    rows=row,
    cols=total_cols,
    shared_yaxes=True,
    x_title=x_title,
    y_title="z-score",
    horizontal_spacing=0.001,
)

for assay_label in sorted(assays):
    assay_df = sex_df[sex_df[ASSAY] == assay_label]

    for track_type in sorted(assay_df[TRACK].unique()):
        sub_df = assay_df[assay_df[TRACK] == track_type]

        y_values = sub_df[coverage_label]
        hovertext = np.array(
            [
                f"{md5sum}:z-score={z_score:.3f} (n={int(count)})"
                for md5sum, z_score, count in zip(
                    sub_df.index,
                    sub_df[coverage_label],
                    sub_df[count_label],
                )
            ]
        )

        female_idx = np.argwhere((sub_df[SEX] == "female").values).flatten()
        male_idx = np.argwhere((sub_df[SEX] == "male").values).flatten()

        fig.add_trace(
            go.Violin(
                y=y_values.values[female_idx],
                name=f"{assay_label},{track_type} ({sub_df.shape[0]})",
                box_visible=True,
                meanline_visible=True,
                points="all",
                line_color="red",
                hovertemplate="%{text}",
                text=hovertext[female_idx],
                side="negative",
            ),
            row=row,
            col=col,
        )

        fig.add_trace(
            go.Violin(
                y=y_values.values[male_idx],
                name=f"{assay_label},{track_type} ({sub_df.shape[0]})",
                box_visible=True,
                meanline_visible=True,
                points="all",
                line_color="blue",
                hovertemplate="%{text}",
                text=hovertext[male_idx],
                side="positive",
            ),
            row=row,
            col=col,
        )

        col += 1

        epirr_scatter_df = sub_df[sub_df[epirr_cat].isin(to_verify_epirr)].sort_values(
            by=epirr_cat
        )
        print(epirr_scatter_df[epirr_cat])
        print(epirr_scatter_df[ASSAY].value_counts())

        if epirr_scatter_df.shape[1] != 0:
            y_values = epirr_scatter_df[coverage_label]

            hovertext = np.array(
                [
                    f"{md5sum},{epirr},{sex}:z-score={z_score:.3f} (n={int(count)})"
                    for md5sum, z_score, count, epirr, sex in zip(
                        epirr_scatter_df.index,
                        epirr_scatter_df[coverage_label],
                        epirr_scatter_df[count_label],
                        epirr_scatter_df[epirr_cat],
                        epirr_scatter_df[SEX],
                    )
                ]
            )
            unknown_sex_cond = epirr_scatter_df[SEX] == "unknown"
            unknown_idx = np.argwhere(unknown_sex_cond.values).flatten()
            known_idx = np.argwhere((~unknown_sex_cond).values).flatten()

            x_pos = list(range(0, len(unknown_idx) + len(known_idx)))

            colors = np.array(
                [epirr_colors[epirr] for epirr in epirr_scatter_df[epirr_cat]]
            )

            if len(unknown_idx) != 0:
                fig.add_trace(
                    go.Scatter(
                        x=x_pos[: len(unknown_idx)],
                        y=y_values.values[unknown_idx],
                        hovertemplate="%{text}",
                        text=hovertext[unknown_idx],
                        marker=dict(color=colors[unknown_idx], symbol="diamond"),
                        mode="markers",
                    ),
                    row=row,
                    col=col,
                )

            if len(known_idx) != 0:
                fig.add_trace(
                    go.Scatter(
                        x=x_pos[len(unknown_idx) :],
                        y=y_values.values[known_idx],
                        hovertemplate="%{text}",
                        text=hovertext[known_idx],
                        marker=dict(color=colors[known_idx], symbol="circle"),
                        mode="markers",
                    ),
                    row=row,
                    col=col,
                )

        else:
            y_values = [0]
            hovertext = ["NO VALUE"]
            fig.add_trace(
                go.Scatter(
                    x_pos=[0],
                    y=y_values,
                    hovertemplate="%{text}",
                    text=hovertext,
                    mode="markers",
                    marker=dict(color=white),
                ),
                row=row,
                col=col,
            )

        col += 1


# global track type distribution (all assays)
for track_type in sorted(sex_df[TRACK].unique()):
    sub_df = sex_df[sex_df[TRACK] == track_type]

    y_values = sub_df[coverage_label]
    hovertext = np.array(
        [
            f"{md5sum,assay}:z-score={z_score:.3f} (n={int(count)})"
            for md5sum, z_score, count, assay in zip(
                sub_df.index,
                sub_df[coverage_label],
                sub_df[count_label],
                sub_df[ASSAY],
            )
        ]
    )
    female_idx = np.argwhere((sub_df[SEX] == "female").values).flatten()
    male_idx = np.argwhere((sub_df[SEX] == "male").values).flatten()

    fig.add_trace(
        go.Violin(
            y=y_values.values[female_idx],
            name=f"{track_type} ({sub_df.shape[0]})",
            box_visible=True,
            meanline_visible=True,
            points="all",
            line_color="red",
            hovertemplate="%{text}",
            text=hovertext[female_idx],
            side="negative",
        ),
        row=row,
        col=col,
    )

    fig.add_trace(
        go.Violin(
            y=y_values.values[male_idx],
            name=f"{track_type} ({sub_df.shape[0]})",
            box_visible=True,
            meanline_visible=True,
            points="all",
            line_color="blue",
            hovertemplate="%{text}",
            text=hovertext[male_idx],
            side="positive",
        ),
        row=row,
        col=col,
    )

    col += 1


# Final graphing

# Update global layout and traces
fig.update_traces(marker=dict(size=1))
fig.update_traces(marker=dict(size=5), selector=dict(mode="markers"))
fig.update_yaxes(
    range=[
        min(sex_df[coverage_label]) - 0.01,
        max(sex_df[coverage_label]) + 0.01,
    ]
)

fig.update_xaxes(tickangle=50)

fig.update_annotations(y=1.5, selector={"text": x_title})

title = "z-score(mean chrY coverage per file) distribution per assay+track type, female/male split"
fig.update_layout(
    title_text=f"{title}",
    showlegend=False,
    autosize=True,
    width=7000,
)

# filename = get_valid_filename(title).replace("_br_", "_")
# html_file = matrix_logdir / f"{filename}.html"
# fig.write_html(html_file)

In [ ]:
raise ValueError("stop here")

#### kde graphs

In [ ]:
# temp_graph_df = merged_pred_df[~merged_pred_df[ASSAY].str.contains("wgb")]
temp_graph_df = merged_pred_df

In [ ]:
x_title = "z-score"
fig = make_subplots(
    rows=1,
    cols=2,
    shared_yaxes=True,
    x_title=x_title,
    y_title="z-score density",
    horizontal_spacing=0.01,
)

x_values = np.linspace(min(temp_graph_df[coverage_label]), 8, 300)

zscore_classifier_acc = []
naive_classifier_answers = []
for assay_label in sorted(assays):
    assay_df = temp_graph_df[temp_graph_df[ASSAY] == assay_label]

    for track_type in sorted(assay_df[TRACK].unique()):
        sub_df = assay_df[assay_df[TRACK] == track_type]

        z_scores = sub_df[coverage_label]

        kde = gaussian_kde(z_scores, bw_method="silverman")
        density = kde(x_values)

        name = f"{assay_label},{track_type} (n={len(z_scores)})"
        hovertext = [f"{assay_label}, {track_type}: {x_val}" for x_val in x_values]

        fig.add_trace(
            go.Scatter(
                x=x_values,
                y=density,
                mode="lines+markers",
                name=name,
                line_color=assay_colors_dict[assay_label],
                hovertemplate="%{text}",
                text=hovertext,
            ),
            row=1,
            col=1,
        )

        if track_type == "fc":
            minimas_idx = find_peaks(density * -1)[0]
            minima = x_values[minimas_idx][0]
            sex_sub_df = sub_df[sub_df[SEX].str.contains("male")]

            delta = 0.05
            lower_bound = minima - delta
            upper_bound = minima + delta
            print(f"{name}: {minima:.3f}" + "\u00b1" + f"{delta:.3f}")

            predicted_male = sex_sub_df[coverage_label] > upper_bound
            predicted_female = sex_sub_df[coverage_label] < lower_bound

            naive_classifier_label = f"naive_sex_pred_delta{0.1}"
            new_df = pd.DataFrame(sex_sub_df[coverage_label])
            new_df[naive_classifier_label] = np.nan

            # Use `.loc` to update the original DataFrame based on conditions
            new_df.loc[
                new_df[coverage_label] > upper_bound, naive_classifier_label
            ] = "male"
            new_df.loc[
                new_df[coverage_label] < lower_bound, naive_classifier_label
            ] = "female"

            naive_classifier_answers.append(new_df)

            # known_male = sex_sub_df[SEX] == "male"
            # known_female = sex_sub_df[SEX] == "female"

            # correct_male = sum(predicted_male & known_male)
            # correct_female = sum(predicted_female & known_female)
            # acc_male = correct_male/known_male.sum()*100
            # acc_female = correct_female/known_female.sum()*100
            # tot_acc = (correct_male+correct_female)/(known_male.sum() + known_female.sum())*100

            # print(f"acc_male: {acc_male:.2f}%")
            # print(f"acc_female: {acc_female:.2f}%")
            # nb_excluded = sum(sex_sub_df[coverage_label] > minima) - sum(predicted_male)
            # nb_excluded += sum(sex_sub_df[coverage_label] < minima) - sum(predicted_female)

            # zscore_classifier_acc.append([name, minima, acc_male, acc_female, tot_acc, nb_excluded])

# df = pd.DataFrame(zscore_classifier_acc, columns=["name", "minima", "acc_male", "acc_female", "acc", "nb_excluded"])
# df.to_csv(matrix_logdir / "chrY_classifier_acc_delta0.1.csv")

naive_classifier_df = pd.concat(naive_classifier_answers).sort_index()
naive_classifier_df.to_csv(
    matrix_logdir / "chrY_naive_classifier_acc_delta0.1_predictions.csv"
)

# global track type distribution (all assays)

# for track_type in sorted(temp_graph_df[TRACK].unique()):
#     sub_df = temp_graph_df[temp_graph_df[TRACK] == track_type]

#     z_scores = sub_df[coverage_label]

#     kde = gaussian_kde(z_scores, bw_method="silverman")
#     density = kde(x_values)

#     fig.add_trace(
#         go.Scatter(
#         x=x_values,
#         y=density,
#         mode="lines+markers",
#         name=f"{track_type} (n={len(z_scores)})",
#         line_color=track_colors_dict[track_type]
#         ),
#         row=1,
#         col=2,
#     )


# # Final graphing

# fig.update_traces(marker=dict(size=1))

# title = "density(z-score(mean chrY)) per assay+track type"
# fig.update_layout(
#     title_text=f"{title}",
#     showlegend=True,
#     autosize=True,
#     width=2000,
#     height=1000,
# )

# fig.show()


# Save graph
# filename = get_valid_filename(title).replace("_br_", "_")
# html_file = matrix_logdir / f"{filename}.html"
# fig.write_html(html_file)

In [ ]:
len(naive_classifier_df)

### Graphs for all assays mixed

In [ ]:
coverage_label = "expected_assay_chrY_z-score"
count_label = f"count_{coverage_label}"

classes = merged_pred_df["True class"].unique()

matrix_logdir = chrY_coverage_file = (
    logs
    / "chrY_coverage_results"
    / "10fold_valid"
    / "z-score"
    / "per_assay"
    / "w-unknown"
)


# confusion matrix for chrY z-score
for threshold in [0, 0.7, 0.9]:
    row = 1
    col = 1
    fig = make_subplots(
        rows=3,
        cols=3,
        shared_yaxes=True,
        x_title="Predicted class (nb of predictions)",
        y_title="z-score for expected assay",
        row_titles=list(classes),
        column_titles=list(classes),
        vertical_spacing=0.08,
        horizontal_spacing=0.01,
    )
    threshold_df = merged_pred_df[merged_pred_df["Max pred"] >= threshold]

    title = (
        f"z-score(mean chrY coverage per file) - (pred>{threshold})<br>w fc/pval, no wgbs"
    )

    filename = get_valid_filename(title).replace("_br_", "_")
    html_file = matrix_logdir / f"{filename}.html"
    png_file = matrix_logdir / f"{filename}.png"
    if png_file.exists() or html_file.exists():
        print(f"{str(html_file)} Already exists, continuing...")
        continue

    for label in classes:
        df_label = threshold_df[threshold_df["True class"] == label]

        # Iterate over each target and add a violin plot for it
        for target in classes:
            sub_df = df_label[df_label["Predicted class"] == target]

            if sub_df.shape[0] == 0:
                y_values = [threshold_df[coverage_label].mean()]  # Minimal synthetic data
                sample_count = 0
                hovertext = ["PLACEHOLDER - NO DATA"]
            else:
                y_values = sub_df[coverage_label]
                hovertext = [
                    f"{md5sum, assay}:(z-score={z_score:.3f} (n={int(count)}), pred={pred:.3f})"
                    for md5sum, pred, z_score, count, assay in zip(
                        sub_df.index,
                        sub_df["Max pred"],
                        sub_df[coverage_label],
                        sub_df[count_label],
                        sub_df[ASSAY],
                    )
                ]

            fig.add_trace(
                go.Violin(
                    y=y_values,
                    name=f"{target} ({sub_df.shape[0]})",
                    box_visible=True,
                    meanline_visible=True,
                    points="all",
                    line_color=COLORS_DICT[target],
                    hovertemplate="%{text}",
                    text=hovertext,
                ),
                row=row,
                col=col,
            )

            # Move to the next subplot position
            col += 1
            if col > 3:
                col = 1
                row += 1

    # Update global layout and traces
    fig.update_traces(marker=dict(size=1))
    fig.update_yaxes(
        range=[
            min(merged_pred_df[coverage_label]) - 0.01,
            max(merged_pred_df[coverage_label]) + 0.01,
        ]
    )

    # Directly using annotations param does not work with make_subplots
    existing_annotations = fig.layout.annotations  # type: ignore
    new_annotation = dict(
        x=1.01,  # Position on the x-axis
        y=0.5,  # Position on the y-axis
        showarrow=False,  # Do not show arrow
        text="Reference class",  # The text you want to display
        xref="paper",  # 'x' coordinate is set in relative coordinates
        yref="paper",  # 'y' coordinate is set in relative coordinates
        xanchor="left",  # Text starts from the left of the x-coordinate
        yanchor="middle",  # Middle aligned vertically
        font=dict(size=16),
        textangle=90,
    )
    updated_annotations = list(existing_annotations) + [new_annotation]

    fig.update_layout(
        title_text=f"{title} (n={threshold_df.shape[0]})",
        showlegend=False,
        annotations=updated_annotations,
    )

    fig.show()

    # fig.write_image(png_file, scale=2)
    # fig.write_html(html_file)

## Merge with global results file

In [ ]:
target_file_dir = logs / "merged_results"
target_file = target_file_dir / "merged_pred_results_all_2.1_chrY_zscores.csv"

target_df = pd.read_csv(target_file, index_col=0, header=0)

In [ ]:
# target_df[SEX].str.contains("male")
sum((target_df[TRACK] == "fc") & (target_df[SEX].str.contains("male|female")))

In [ ]:
updated_target_df = target_df.merge(
    naive_classifier_df,
    how="outer",
    left_index=True,
    right_index=True,
    suffixes=("", "_DROP"),
).sort_index()

In [ ]:
same_cols = []
for column in updated_target_df.columns:
    if column.endswith("_DROP"):
        same_cols.append(column[:-5])

In [ ]:
for column in same_cols:
    assert np.isclose(
        0, (updated_target_df[column] - updated_target_df[f"{column}_DROP"]).sum()
    ), f"{column} not equal"

In [ ]:
updated_target_df = updated_target_df.drop(
    columns=[column for column in updated_target_df.columns if column.endswith("_DROP")]
)

In [ ]:
updated_target_df.to_csv(
    target_file_dir / "merged_pred_results_all_2.1_chrY_zscores_w_naive.csv"
)